### Introduction/Business Problem

This project is aimed to explore and suggest the best regions for people who want to start a vending machines business in Boston(US). In retail, the most important contributor to success is location. Usually, business owners want their machines in places that have plenty of foot traffic, such as malls, large office complexes, schools, airports, in front of stores, etc. 

Geographic segmentation will help us segmenting the market based on location. Segmentation could be done by zip, city, borough, etc. depending on the scope of the business. We are making an assumption that having the vinding machines business this could be done by neighbourhood.

### Data

First, we need a list of neighborhoods in Boston before querying. We will use the list of neighborhoods available on the https://data.boston.gov/ page and filter out all the neighborhoods: http://bostonopendata-boston.opendata.arcgis.com/datasets/3525b0ee6e6b427f9aab5d0a1d0a1a28_0.csv 

The example of data as follows:

|OBJECTID | Name      | Acres        | Neighborhood_ID | SqMiles | ShapeSTArea       | ShapeSTLength     |
|:-------:|:---------:|:------------:|:---------------:|:-------:|:-----------------:|:-----------------:|
|27	      |Roslindale |	1605.5682375 | 15              |	2.51 |	69938272.9255705 |	53563.9125970566 |

Next, with the list of regions, we will get the latitudes and longitudes of the neighborhoods with geocoder (or other services if geocoder service is not available). Having the coordinates of the neighborhoods, we can then query Foursquare to explore the amount and variety of the foot traffic places in the neighborhood and suggest the most fitting ones.

### Methodology

Data of neighborhoods in Boston was retrieved from the CSV file obtained from the opened recourses.
Then the data was preprocessed by removing redundant fields as well as cleaned out of bad data.

The latitude and longitude values for each neighborhood were obtained by converting an address(neighborhood name and state) with the help of Nominatim library.

Then we iterated through each neighbourhood and queried the Foursquare API to get popular venues nearby. The venues were then turned into one-hot encoding and normalized to be clustered by k-means.

With k-means clustering, the neighbourhoods were clustered into 4 clusters and finally the clustering was displayed on the Folium map.

### Results

The second and fourth clusters contain mostly restaurants and cafes which have enough crowd but low demand of fast snacks. First cluster is better match but still has lots of restaurants which decreases the demand of snacks. The third cluster, in our opinion, is the best fit because besides the cafes and restaurants it has venues with plenty of foot traffic, such as parks, pubs, supermarkets, gyms.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/dinadraguzya/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.3.0               |        py37_1001         491 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         589 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/osx-64::altair-2.3.0-py37_1001
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             co

#### Creating dataset

Let's retrieve the neighborhoods data and put it into the dataframe

In [2]:
raw_data = pd.read_csv("http://bostonopendata-boston.opendata.arcgis.com/datasets/3525b0ee6e6b427f9aab5d0a1d0a1a28_0.csv")
df = raw_data[['Name']]
df.columns = ['Neighborhood']
df.head()

,Neighborhood
0,Roslindale
1,Jamaica Plain
2,Mission Hill
3,Longwood
4,Bay Village


Now, we are converting an address into latitude and longitude values for each neighborhood

In [3]:
latitudes = []
longitudes = []

for row in df['Neighborhood']:
    address = '{}, MA'.format(row)
    geolocator = Nominatim(user_agent="boston_explorer")
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
    
lats = pd.DataFrame(latitudes, columns=['Latitude'])
longs = pd.DataFrame(longitudes, columns=['Longitude'])
df = pd.concat([df, lats, longs], axis=1)
df.head()

,Neighborhood,Latitude,Longitude
0,Roslindale,42.291209,-71.124497
1,Jamaica Plain,42.309820,-71.120330
2,Mission Hill,42.332926,-71.103214
3,Longwood,42.341706,-71.109959
4,Bay Village,42.350011,-71.066948


For some reason, the coordinates we're getting for Leather District are incorrect and pointing us to some place at South Africa so let's remove this row from the dataframe to rid off the bad data

In [4]:
df = df[~df['Neighborhood'].str.contains("Leather District")].reset_index(drop=True)
df.head()

,Neighborhood,Latitude,Longitude
0,Roslindale,42.291209,-71.124497
1,Jamaica Plain,42.309820,-71.120330
2,Mission Hill,42.332926,-71.103214
3,Longwood,42.341706,-71.109959
4,Bay Village,42.350011,-71.066948


In [6]:
# create map of Boston using latitude and longitude values
address = 'Boston, MA'
geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them

In [7]:
CLIENT_ID = 'DPIER3JTIXV0DOCSHZZ0FTTEZM3SF0ZPLBYVTVLU2QRODVZE'
CLIENT_SECRET = 'ZSYWWUMBPTRRZ3DGCADBG3V5VFKOV2TYXP1U1M0DFTAVQ0M1'
VERSION = '20180605'

Let's create a function to process all neighborhoods in Toronto having the limit of number of venues returned by Foursquare API equalls to 100 and radius equals to 1000

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
boston_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Roslindale
Jamaica Plain
Mission Hill
Longwood
Bay Village
Chinatown
North End
Roxbury
South End
Back Bay
East Boston
Charlestown
West End
Beacon Hill
Downtown
Fenway
Brighton
West Roxbury
Hyde Park
Mattapan
Dorchester
South Boston Waterfront
South Boston
Allston
Harbor Islands


In [10]:
print(boston_venues.shape)
boston_venues.head()

(1681, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Roslindale,42.291209,-71.124497,Peters Hill,42.293617,-71.128063,Scenic Lookout
1,Roslindale,42.291209,-71.124497,Roslindale House Of Pizza,42.287989,-71.126549,Pizza Place
2,Roslindale,42.291209,-71.124497,Delfino’s,42.287106,-71.129470,Italian Restaurant
3,Roslindale,42.291209,-71.124497,Roslindale Village Farmers Market,42.286534,-71.128509,Farmers Market
4,Roslindale,42.291209,-71.124497,Fornax Bread Company,42.286171,-71.129760,Bakery


In [11]:
boston_venues['Venue Category'].unique()

array(['Scenic Lookout', 'Pizza Place', 'Italian Restaurant',
       'Farmers Market', 'Bakery', 'American Restaurant', 'Plaza',
       'Department Store', 'Pool', 'Coffee Shop', 'Restaurant', 'Bar',
       'Rental Car Location', 'Sandwich Place', 'Ice Cream Shop',
       'Cheese Shop', 'Grocery Store', 'Indian Restaurant', 'Donut Shop',
       'Pub', 'Discount Store', 'Pharmacy', 'Yoga Studio',
       'Convenience Store', 'Chinese Restaurant', 'Deli / Bodega',
       'Liquor Store', 'Cosmetics Shop', 'Trail', 'Accessories Store',
       'Pet Store', 'Theater', 'Park', 'Thai Restaurant', 'Record Shop',
       'Mexican Restaurant', 'Bookstore', 'Seafood Restaurant',
       'Thrift / Vintage Store', 'New American Restaurant',
       'Asian Restaurant', 'Miscellaneous Shop', 'Art Gallery',
       'Taco Place', 'Middle Eastern Restaurant', 'Veterinarian',
       'Burrito Place', 'Harbor / Marina', 'Lake', 'Flower Shop',
       'Laundromat', 'Gastropub', 'Art Museum', 'Caribbean Restaurant'

#### Analyze Each Neighborhood in Boston

First, let's use one-hot encoding to convert the categorical venue category to a binary classifier column for each venue category

In [12]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhoods'] = boston_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Casino,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pub,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Roslindale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Roslindale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [13]:
boston_grouped = boston_onehot.groupby('Neighborhoods').mean().reset_index()
boston_grouped.head()

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Casino,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pub,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allston,0.0,0.00,0.012048,0.0,0.000000,0.0,0.0,0.0,0.00,0.012048,0.012048,0.0,0.00,0.036145,0.000000,0.036145,0.012048,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.012048,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.012048,0.012048,0.0,0.012048,0.000000,0.0,0.0,0.0,0.024096,0.0,0.0,0.0,0.0,0.048193,0.00,0.0,0.0,0.0,0.012048,0.00,0.024096,0.0,0.00,0.012048,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.000000,0.012048,0.000000,0.024096,0.0,0.00,0.0,0.000000,0.012048,0.000000,0.012048,0.012048,0.012048,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.012048,0.012048,0.000000,0.00,0.012048,0.012048,0.012048,0.00,0.0,0.024096,0.0,0.00,0.000000,0.0,0.00,0.000000,0.012048,0.012048,0.024096,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.012048,0.012048,0.0,0.0,0.00,0.012048,0.00,0.00,0.012048,0.012048,0.012048,0.0,0.01

Let's print each neighborhood along with the top 5 most common venues

In [14]:
num_top_venues = 5

for hood in boston_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                venue  freq
0         Pizza Place  0.06
1   Korean Restaurant  0.05
2  Chinese Restaurant  0.05
3                 Bar  0.04
4              Bakery  0.04


----Back Bay----
                 venue  freq
0          Coffee Shop  0.07
1                  Spa  0.04
2  American Restaurant  0.04
3   Italian Restaurant  0.04
4                Hotel  0.04


----Bay Village----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.06
2                 Spa  0.06
3  Chinese Restaurant  0.05
4             Theater  0.04


----Beacon Hill----
                 venue  freq
0  American Restaurant  0.05
1          Coffee Shop  0.04
2   Italian Restaurant  0.04
3        Historic Site  0.04
4            Hotel Bar  0.04


----Brighton----
               venue  freq
0        Pizza Place  0.08
1        Coffee Shop  0.05
2                Pub  0.05
3  Convenience Store  0.05
4             Bakery  0.04


----Charlestown----
            venue  freq
0         

Let's put that into a pandas dataframe

In [50]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = boston_grouped['Neighborhoods']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Pizza Place,Korean Restaurant,Chinese Restaurant,Bar,Bakery,Japanese Restaurant,Gastropub,Liquor Store,Thai Restaurant,Rock Club
1,Back Bay,Coffee Shop,Seafood Restaurant,Hotel,Spa,American Restaurant,Italian Restaurant,Gym / Fitness Center,Ice Cream Shop,Chocolate Shop,Sporting Goods Shop
2,Bay Village,Coffee Shop,Spa,Italian Restaurant,Chinese Restaurant,Theater,Asian Restaurant,Gym,American Restaurant,Dessert Shop,Pizza Place
3,Beacon Hill,American Restaurant,Coffee Shop,Historic Site,Italian Restaurant,Hotel Bar,Restaurant,Seafood Restaurant,Pizza Place,Steakhouse,Hotel
4,Brighton,Pizza Place,Convenience Store,Coffee Shop,Pub,Café,Bakery,Donut Shop,Chinese Restaurant,Grocery Store,Sushi Restaurant


### Cluster Neighborhoods in Boston

Let's run k-means to cluster the neighborhood into 4 clusters

In [51]:
# set number of clusters
kclusters = 4

boston_grouped_clustering = boston_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 2, 0, 2, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

boston_merged = df

# merge boston_grouped with df to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

boston_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roslindale,42.291209,-71.124497,2,Pizza Place,Grocery Store,American Restaurant,Yoga Studio,Donut Shop,Bar,Pharmacy,Sandwich Place,Italian Restaurant,Farmers Market
1,Jamaica Plain,42.309820,-71.120330,0,Bakery,Park,Coffee Shop,Pizza Place,Yoga Studio,Pub,Bookstore,Thrift / Vintage Store,Art Gallery,American Restaurant
2,Mission Hill,42.332926,-71.103214,0,Café,Park,Donut Shop,Sandwich Place,Coffee Shop,Pizza Place,Art Museum,Sushi Restaurant,Italian Restaurant,Hotel
3,Longwood,42.341706,-71.109959,0,Café,Sandwich Place,Park,Coffee Shop,American Restaurant,Bakery,Greek Restaurant,Gym,Thai Restaurant,Mexican Restaurant
4,Bay Village,42.350011,-71.066948,0,Coffee Shop,Spa,Italian Restaurant,Chinese Restaurant,Theater,Asian Restaurant,Gym,American Restaurant,Dessert Shop,Pizza Place


Finally, let's visualize the resulting clusters

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighborhood'], boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### First cluster

In [54]:
boston_merged.loc[boston_merged['Cluster Labels'] == 0, boston_merged.columns[[0] + list(range(4, boston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Jamaica Plain,Bakery,Park,Coffee Shop,Pizza Place,Yoga Studio,Pub,Bookstore,Thrift / Vintage Store,Art Gallery,American Restaurant
2,Mission Hill,Café,Park,Donut Shop,Sandwich Place,Coffee Shop,Pizza Place,Art Museum,Sushi Restaurant,Italian Restaurant,Hotel
3,Longwood,Café,Sandwich Place,Park,Coffee Shop,American Restaurant,Bakery,Greek Restaurant,Gym,Thai Restaurant,Mexican Restaurant
4,Bay Village,Coffee Shop,Spa,Italian Restaurant,Chinese Restaurant,Theater,Asian Restaurant,Gym,American Restaurant,Dessert Shop,Pizza Place
5,Chinatown,Chinese Restaurant,Coffee Shop,Sandwich Place,Italian Restaurant,Asian Restaurant,American Restaurant,Sushi Restaurant,Theater,Park,Historic Site
8,South End,Pizza Place,Hotel,Coffee Shop,Shipping Store,Gym,Hockey Arena,American Restaurant,Museum,Burger Joint,Sandwich Place
9,Back Bay,Coffee Shop,Seafood Restaurant,Hotel,Spa,American Restaurant,Italian Restaurant,Gym / Fitness Center,Ice Cream Shop,Chocolate Shop,Sporting Goods Shop
10,East Boston,Mexican Restaurant,Park,Latin American Restaurant,Italian Restaurant,Pizza Place,Seafood Restaurant,Café,Sandwich Place,Restaurant,Liquor Store
12,West End,Italian Restaurant,Bakery,Park,Pizza Place,Coffee Shop,Seafood Restaurant,Historic Site,Hotel,Hotel Bar,Steakhouse
13,Beacon Hill,American Restaurant,Coffee Shop,Historic Site,Italian Restaurant,Hotel Bar,Restaurant,Seafood Restaurant,Pizza Place,Steakhouse,Hotel


#### Second cluster

In [55]:
boston_merged.loc[boston_merged['Cluster Labels'] == 1, boston_merged.columns[[0] + list(range(4, boston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Harbor Islands,Lake,Yoga Studio,Dive Bar,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dumpling Restaurant


#### Third cluster

In [56]:
boston_merged.loc[boston_merged['Cluster Labels'] == 2, boston_merged.columns[[0] + list(range(4, boston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roslindale,Pizza Place,Grocery Store,American Restaurant,Yoga Studio,Donut Shop,Bar,Pharmacy,Sandwich Place,Italian Restaurant,Farmers Market
7,Roxbury,Pizza Place,Donut Shop,Park,Italian Restaurant,Pharmacy,Sandwich Place,Supermarket,State / Provincial Park,Skating Rink,Liquor Store
11,Charlestown,Park,Donut Shop,Gastropub,Café,Pizza Place,History Museum,Convenience Store,Pub,Coffee Shop,National Park
16,Brighton,Pizza Place,Convenience Store,Coffee Shop,Pub,Café,Bakery,Donut Shop,Chinese Restaurant,Grocery Store,Sushi Restaurant
17,West Roxbury,Pizza Place,Pharmacy,Liquor Store,Convenience Store,Thai Restaurant,Donut Shop,Park,American Restaurant,Grocery Store,Discount Store
18,Hyde Park,Pizza Place,Pharmacy,Park,American Restaurant,Grocery Store,Bar,Coffee Shop,Discount Store,Sandwich Place,Donut Shop
19,Mattapan,Caribbean Restaurant,Pizza Place,Home Service,BBQ Joint,Rental Car Location,Scenic Lookout,Donut Shop,Shopping Mall,Pharmacy,Liquor Store
20,Dorchester,Pharmacy,Fast Food Restaurant,Pizza Place,Liquor Store,Platform,Bank,Park,Discount Store,Sandwich Place,Mobile Phone Shop
21,South Boston Waterfront,Pizza Place,Bar,Donut Shop,Coffee Shop,Park,Convenience Store,Gym,Italian Restaurant,Sandwich Place,Juice Bar
22,South Boston,Pizza Place,Bar,Donut Shop,Coffee Shop,Park,Convenience Store,Gym,Italian Restaurant,Sandwich Place,Juice Bar


#### Fourth cluster

In [57]:
boston_merged.loc[boston_merged['Cluster Labels'] == 3, boston_merged.columns[[0] + list(range(4, boston_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North End,Italian Restaurant,Bakery,Seafood Restaurant,Park,Pizza Place,Café,Coffee Shop,Historic Site,Sandwich Place,Market
